Q1: K-Fold Cross Validation for Multiple Linear Regression (Least Square Error Fit)

Load the dataset and Implement 5- fold cross validation for multiple linear regression (using least square error fit).


a) Divide the dataset into input features (all columns except price) and output variable
(price)

b) Scale the values of input features.

c) Divide input and output features into five folds.

d) Run five iterations, in each iteration consider one-fold as test set and remaining four sets as training set. Find the beta (𝛽) matrix, predicted values, and R2_score

e) Use the best value of (𝛽) matrix (for which R2_score is maximum), to train the regressor for 70% of data, and test the data for remaining 30% data




In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# load data
df = pd.read_csv("USA_Housing.csv")

# split X and y (drop Price + Address)
X = df.drop(['Price', 'Address'], axis=1).values
y = df['Price'].values.reshape(-1, 1)

# scale X
X = StandardScaler().fit_transform(X)

# 5-fold CV
folds = np.array_split(np.arange(len(X)), 5)
best_r2, best_b = -1, None

for i in range(5):
    te_idx = folds[i]
    tr_idx = np.concatenate([folds[j] for j in range(5) if j != i])

    X_tr, y_tr = X[tr_idx], y[tr_idx]
    X_te, y_te = X[te_idx], y[te_idx]

    X_tr_b = np.c_[np.ones((X_tr.shape[0], 1)), X_tr]
    X_te_b = np.c_[np.ones((X_te.shape[0], 1)), X_te]

    b = np.linalg.inv(X_tr_b.T @ X_tr_b) @ X_tr_b.T @ y_tr
    y_pred = X_te_b @ b
    r2 = r2_score(y_te, y_pred)

    if r2 > best_r2:
        best_r2, best_b = r2, b

# 70/30 split using best beta
n = int(0.7 * len(X))
X_tr, y_tr = X[:n], y[:n]
X_te, y_te = X[n:], y[n:]

X_tr_b = np.c_[np.ones((X_tr.shape[0], 1)), X_tr]
X_te_b = np.c_[np.ones((X_te.shape[0], 1)), X_te]

y_pred = X_te_b @ best_b

print("Eval with best βeta on 70/30 split ")
print("R2 (test):", r2_score(y_te, y_pred))
print("Best βeta:", best_b.flatten())


>>> Eval with best β on 70/30 split <<<
R2 (test): 0.9177860344386396
Best β: [1.23144707e+06 2.29921558e+05 1.64523054e+05 1.19737507e+05
 1.12425659e+03 1.51317802e+05]


Q2)Concept of Validation set for Multiple Linear Regression (Gradient Descent Optimization)

Consider the same dataset of Q1, rather than dividing the dataset into five folds, divide the dataset into training set (56%), validation set (14%), and test set (30%).

Consider four different values of learning rate i.e. {0.001,0.01,0.1,1}. Compute the values of regression coefficients for each value of learning rate after 1000 iterations.

For each set of regression coefficients, compute R2_score for validation and test set and find the best value of regression coefficients

In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Load dataset
df = pd.read_csv("USA_Housing.csv")

# Features (X) and target (y)
X = df.drop(["Price", "Address"], axis=1).values
y = df["Price"].values.reshape(-1, 1)

# Scale X
X = StandardScaler().fit_transform(X)

# Add bias term
X = np.c_[np.ones((X.shape[0], 1)), X]

# Split: 56% train, 14% validation, 30% test
n = len(X)
n_train = int(0.56 * n)
n_val = int(0.14 * n)

X_train, y_train = X[:n_train], y[:n_train]
X_val, y_val = X[n_train:n_train+n_val], y[n_train:n_train+n_val]
X_test, y_test = X[n_train+n_val:], y[n_train+n_val:]

# Gradient Descent
def gradient_descent(X, y, lr=0.01, iters=1000):
    m, n = X.shape
    beta = np.zeros((n, 1))
    for i in range(iters):
        y_pred = X @ beta
        grad = (2/m) * (X.T @ (y_pred - y))
        beta -= lr * grad

        if np.any(np.isnan(beta)) or np.any(np.abs(beta) > 1e10):
            print(f" Diverged at iteration {i} with lr={lr}")
            return None
    return beta

# Try different learning rates
learning_rates = [0.001, 0.01, 0.1, 1]
results = []

print("Gradient Descent Results")
for lr in learning_rates:
    beta = gradient_descent(X_train, y_train, lr=lr, iters=1000)
    if beta is None:
        continue

    y_val_pred = X_val @ beta
    y_test_pred = X_test @ beta

    r2_val = r2_score(y_val, y_val_pred)
    r2_test = r2_score(y_test, y_test_pred)

    print(f"\nLearning Rate = {lr}")
    print("Beta matrix:", beta.ravel())
    print("Predicted values:", y_test_pred[:10].ravel())
    print("R squared score (validation):", r2_val)
    print("R squared score (test):", r2_test)

    results.append((beta, r2_val, r2_test, lr))

# Best beta (by validation R²)
best_beta, best_r2_val, best_r2_test, best_lr = max(results, key=lambda x: x[1])

print("Best Model (Gradient Descent)")
print("Best learning rate:", best_lr)
print("Best Beta matrix:", best_beta.ravel())
print("Predicted values:", (X_test @ best_beta)[:10].ravel())
print("Best Rsquared score (validation):", best_r2_val)
print("Best Rsquared score (test):", best_r2_test)


Gradient Descent Results

Learning Rate = 0.001
Beta matrix: [1066510.29161669  201673.60765789  138729.76331944   97593.06609172
   17538.00033914  128063.29202692]
Predicted values: [ 733242.97677926 1088414.78038638 1727824.68957946 1507945.34777326
 1198155.77535322  739444.49794598 1107722.8607091  1764851.31139852
 1077045.20310253 1247041.45021679]
R squared score (validation): 0.6546707136163156
R squared score (test): 0.6905210120949743

Learning Rate = 0.01
Beta matrix: [ 1.23244778e+06  2.31682562e+05  1.63635182e+05  1.19023941e+05
 -2.73692630e+02  1.50705963e+05]
Predicted values: [ 860800.28623447 1282999.27039571 2017206.68287786 1710879.94315546
 1418667.99853655  845333.65120904 1264104.58497475 2029204.11936143
 1238914.86301228 1455533.41872148]
R squared score (validation): 0.9151039824097136
R squared score (test): 0.917477093208639

Learning Rate = 0.1
Beta matrix: [ 1.23244775e+06  2.31682635e+05  1.63635272e+05  1.19025219e+05
 -2.74956844e+02  1.50705906e+05]


Q3)Load the dataset with following column names ["symboling", "normalized_losses", "make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels", "engine_location", "wheel_base", "length", "width", "height", "curb_weight", "engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"] and replace all ? values with NaN

Replace all NaN values with central tendency imputation. Drop the rows with NaN values in price column

There are 10 columns in the dataset with non-numeric values. Convert these values to numeric values using following scheme:
 (i) For “num_doors” and “num_cylinders”: convert words (number names) to figures for e.g., two to 2

 (ii) For "body_style", "drive_wheels": use dummy encoding scheme

 (iii) For “make”, “aspiration”, “engine_location”,fuel_type: use label encoding scheme

 (iv) For fuel_system: replace values containing string pfi to 1 else all values to 0.

 (v) For engine_type: replace values containing string ohc to 1 else all values to 0.

Divide the dataset into input features (all columns except price) and output variable (price). Scale all input features.
Train a linear regressor on 70% of data (using inbuilt linear regression function of Python) and test its performance on remaining 30% of data.
Reduce the dimensionality of the feature set using inbuilt PCA decomposition and then again train a linear regressor on 70% of reduced data (using inbuilt linear regression function of Python). Does it lead to any performance improvement on test set.



In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score


cols = ["symboling","normalized_losses","make","fuel_type","aspiration",
        "num_doors","body_style","drive_wheels","engine_location","wheel_base",
        "length","width","height","curb_weight","engine_type","num_cylinders",
        "engine_size","fuel_system","bore","stroke","compression_ratio",
        "horsepower","peak_rpm","city_mpg","highway_mpg","price"]

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
df = pd.read_csv(url, names=cols)


df = df.replace("?", np.nan)

# 3) convert cols to numeric where possible
num_cols = ["normalized_losses","wheel_base","length","width","height",
            "curb_weight","engine_size","bore","stroke","compression_ratio",
            "horsepower","peak_rpm","city_mpg","highway_mpg","price"]

for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 4) impute missing values with mean (central tendency)
for c in num_cols:
    df[c] = df[c].fillna(df[c].mean())

# drop rows where price is NaN
df = df.dropna(subset=["price"])

# 5) convert text to num
# a) num_doors + num_cylinders
door_map = {"two":2,"four":4}
cyl_map  = {"two":2,"three":3,"four":4,"five":5,"six":6,"eight":8,"twelve":12}
df["num_doors"] = df["num_doors"].map(door_map).fillna(4)   # assume 4 if NaN
df["num_cylinders"] = df["num_cylinders"].map(cyl_map)

# b) dummy encode body_style, drive_wheels
df = pd.get_dummies(df, columns=["body_style","drive_wheels"], drop_first=True)

# c) label encode make, aspiration, engine_location, fuel_type
enc = LabelEncoder()
for c in ["make","aspiration","engine_location","fuel_type"]:
    df[c] = enc.fit_transform(df[c])

# d) fuel_system: pfi=1 else 0
df["fuel_system"] = df["fuel_system"].apply(lambda x: 1 if "pfi" in str(x) else 0)

# e) engine_type: ohc=1 else 0
df["engine_type"] = df["engine_type"].apply(lambda x: 1 if "ohc" in str(x) else 0)

# 6) split X, y
X = df.drop("price", axis=1).values
y = df["price"].values

# scale X
sc = StandardScaler()
X = sc.fit_transform(X)

# 7) 70/30 split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3, random_state=42)


lr = LinearRegression()
lr.fit(X_tr, y_tr)
y_pr = lr.predict(X_te)

mse1 = mean_squared_error(y_te, y_pr)
r21  = r2_score(y_te, y_pr)


pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X)

Xp_tr, Xp_te, yp_tr, yp_te = train_test_split(X_pca, y, test_size=0.3, random_state=42)

lr2 = LinearRegression()
lr2.fit(Xp_tr, yp_tr)
yp_pr = lr2.predict(Xp_te)

mse2 = mean_squared_error(yp_te, yp_pr)
r22  = r2_score(yp_te, yp_pr)


print(f"Original Model result of  {mse1}  R squared is : {r21}")
print(f"PCA Model result of    {mse2}  R squared is : {r22}")


Original Model result of  13422229.591732549  R squared is : 0.804442243576259
PCA Model result of    17154268.253029242  R squared is : 0.7500675882701553
